In [8]:
%pip install pandas 

Note: you may need to restart the kernel to use updated packages.


In [14]:
import json
import pandas as pd
import requests


path = 'testset_combined.csv'

df = pd.read_csv(path)

# Anzeigen der ersten paar Zeilen der DataFrame


def update_row(row):
    chatReqObj = {
        "id": "someId",
        "messages": [{"role": "User", "content": row['question']}],
    }

    # make http post request with reqObj to https://ellie.stein.solutions/api/chat
    #chatResp = requests.post('https://ellie.stein.solutions/api/chat', json=chatReqObj)
    chatResp = requests.post('http://localhost:3000/api/chat', json=chatReqObj)

    retrieverReqObj = {
        "messages": [],
        "newMessage": row['question']
    }

    retrieverResp = requests.post('http://localhost:3000/api/chat/retrieve', json=retrieverReqObj)
    #retrieverResp = requests.post('https://ellie.stein.solutions/api/chat/retrieve', json=retrieverReqObj)


    # read response obj of chatResp

    row["answer"] = chatResp.content

    row["generated_contexts"] = row["contexts"]

    contexts = [doc["pageContent"] for doc in retrieverResp.json()]
    row["contexts"] = json.dumps(contexts)

    return row

df = df.apply(update_row, axis=1)

print(df.head())

df.to_csv('evalSet.csv', index=False)
    

SSLError: HTTPSConnectionPool(host='localhost', port=3000): Max retries exceeded with url: /api/chat (Caused by SSLError(SSLError(1, '[SSL] record layer failure (_ssl.c:1000)')))